In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as MSE
from bayes_opt import BayesianOptimization
from sklearn import metrics

In [ ]:
df = pd.read_csv('../input/avacado-price-prediction/Avocado.csv')
df.drop(['Unnamed: 0'],axis =1,inplace=True )
df.head()

In [ ]:
df.info() 

We have to arrange ' Region' and 'Date ' columns.( object to dummy) 

In [ ]:
# ENCODING

def ordinal_encoding(df,column,ordering) :
    df =df.copy()
    df[column] = df[column].apply(lambda x : ordering.index(x))
    return df

In [ ]:
date_ordering = sorted(df['Date'].unique())

In [ ]:
df = ordinal_encoding(df,'Date',date_ordering)

In [ ]:
df = pd.get_dummies(df,columns=['region'])

In [ ]:
df = pd.get_dummies(df,columns=['type'])

In [ ]:
# After that process our new df(data) 

df.head()

Preprocessing Model

In [ ]:
reg_X = df.drop(['AveragePrice'],axis=1)
reg_y = df['AveragePrice']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reg_X, reg_y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

coeff_df = pd.DataFrame(regressor.coef_, reg_X.columns, columns=['Coefficient'])
coeff_df

y_pred = regressor.predict(X_test)
y_pred

df_pred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_pred.head()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Root Mean Squared Error: 0.26 is too high for this dataset.Thus we have to find another way.Xgboost algorithm will be usefull !

In [ ]:
## XGBOOST ALGORITHM MODELLING 

In [ ]:
model1 = xgb.XGBRegressor(nthread=10)

In [ ]:
xgb_r = xgb.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 123)
train_X, test_X, train_y, test_y = train_test_split(reg_X, reg_y,
                      test_size = 0.3, random_state = 123)
xgb_r.fit(train_X, train_y)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.35,
                max_depth = 6, alpha = 10, n_estimators = 300)

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

In [ ]:
xgb_r.fit(train_X, train_y)
  
# Predict the model
pred = xgb_r.predict(test_X)
  
# RMSE Computation
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

RMSE :  0.183120 is cool but not enough ! So , i used bayesian optimization for get optimum parameters 

## BAYESIAN OPTIMIZATION 

In [ ]:
def xgbc_cv(max_depth,learning_rate,n_estimators,reg_alpha):
    
    estimator_function = xgb.XGBRegressor(max_depth=int(max_depth),
                                           learning_rate= learning_rate,
                                           n_estimators= int(n_estimators),
                                           reg_alpha = reg_alpha,
                                           nthread = -1,
                                          
                                           seed = seed)
        # Fit the estimator
    estimator_function.fit(X_train,y_train)
    

    return 1/MSE(y_test,estimator_function.predict(X_test))



gp_params = {"alpha": 1e-10}
seed = 112
hyperparameter_space = {
    'max_depth': (3, 15),
    'learning_rate': (0, 1),
    'n_estimators' : (50,600),
    'reg_alpha': (0,1)
}

xgbcBO = BayesianOptimization(f = xgbc_cv, 
                             pbounds =  hyperparameter_space,
                             random_state = seed,
                             verbose = 10)


xgbcBO.maximize(init_points=2,n_iter=6,acq='ucb', kappa= 3, **gp_params)

In [ ]:
optimum_parameter=xgbcBO.max
optimum_parameter["params"]
## that codes give us optimum parameters for xgboost

In [ ]:
xg_reg1 = xgb.XGBRegressor(learning_rate = 0.2946386696029385,
 max_depth= 10,
 n_estimators =571,
 reg_alpha =  0.8663005771251282)

## xgboost algorithm with optimum parameters

In [ ]:
xg_reg1.fit(X_train,y_train)

preds1 = xg_reg1.predict(X_test)

In [ ]:
rmse = np.sqrt(MSE(y_test, preds1))
print("RMSE: %f" % (rmse))
mse= rmse**2
print("MSE: %f" % (mse))

In [ ]:
metrics.r2_score(y_test,preds1)